<a href="https://colab.research.google.com/github/eeyvee-0x4d/cs-thesis/blob/main/Notebook/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Clone repository</h1>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()


df = pd.read_csv('/gdrive/MyDrive/vaccination_all_tweets.csv')

In [ ]:
df = df.drop(columns=['id', 'user_description', 'user_created', 'user_followers', 'user_friends', 'user_favourites', 'user_verified', 'hashtags', 'source', 'retweets', 'favorites', 'is_retweet'])
df.to_csv(path_or_buf='/content/tweets.csv', index=False)

In [ ]:
txtfile = open('/content/locations.txt', 'r')
locations = txtfile.read().splitlines()

df = pd.read_csv('/content/tweets.csv')
tweets = df[df['user_location'].isin(locations)]
tweets.to_csv(path_or_buf='/content/tweets.csv', index=False)

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/eeyvee-0x4d/cs-thesis thesis
%cd thesis
!ls

Cloning into 'thesis'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (282/282), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 282 (delta 71), reused 196 (delta 46), pack-reused 0
Receiving objects: 100% (282/282), 42.24 MiB | 3.17 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/thesis
Dataset  Notebook  README.md  StreamingAPI


In [ ]:
import re
import pandas as pd

In [ ]:
df = pd.read_csv('/content/thesis/Dataset/Astra-December.csv')
print(df)

   Sentiment                                               Text
0   Negative    oh my god sakita sa akong bukton uy astrazeneca
1   Negative  nakakapanghina ng katawan ang vaccine nagkaron...
2   Positive  second dose done heart astrazeneca dbcvaccinat...
3   Negative  same lang din side effect ng first at second d...
4   Positive  booster shot done resbakuna aztrazeneca-pfizer...
5   Positive  got boosted today  astrazeneca covid booster b...
6   Positive  just got boosted let us fight covid variants t...
7   Positive  booster done astrazeneca resbakuna booster htt...
8   Positive  yay mama and papa got their first shot today r...
9   Positive  doing my civic and moral duty to protect mysel...
10  Positive  just got boosted let us fight covid variants  ...
11  Positive  booster done astrazeneca resbakuna booster htt...
12  Negative  ang sakit ng buong katawan ko muscles and butu...
13  Negative  got my first dose yan tapang ka kanina lagnat ...
14  Positive  done second dose salamat s

<h1>Text Preprocessing</h1>
<ul>
  <li>Import dataset</li>
  <li>Remove urls</li>
  <li>Remove special characters</li>
  <li>Convert text data to lowercase</li>
</ul>

In [ ]:
df = pd.read_csv('/content/thesis/Dataset/Astra-December.csv') #read csv

df_shape = df.shape # (row, column)

# remove urls, remove special chars, conver to lowercase
for i in range(df_shape[0]):
  string = re.sub(r'http\S+', '', df.at[i, 'Text']).lower()
  df.at[i, 'Text'] = re.sub(r'\n', '', string)

print(df)


<h1>Natural Language Toolkit NLTK</h1>
<p>
Nltk will be used to preprocess to corpus.
</p>

In [46]:
import nltk

from nltk.stem import *
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

<h1>Stemming words using Porter Stemmer</h1>
<p>
Apply porter stemmer to each tokens first then rebuild the tokens into sentence.
</p>
<hr>
<h1>Stop words removal</h1>
<p>
Remove stop words in english and tagalog.
</p>

In [64]:
corpus = df["Text"]

stemmer = PorterStemmer() # Porter Stemmer

stopwords_eng = set(stopwords.words('english')) # English stopwords
filtered_sentence = []

for i in range(len(corpus)):
  document = corpus[i]
  tokens = nltk.word_tokenize(document)

  stemmed_tokens = [stemmer.stem(token) for token in tokens] # stem each words
  filtered_sentence = [token for token in stemmed_tokens if not token in stopwords_eng] # remove english stopwords
  # remove tagalog stopwords

  document = " ".join(filtered_sentence)
  corpus[i] = document

<h1>Create n-grams from 1-4 ?</h1>
<p>
Create n-grams from 1 to 4 and evaluate each.
</p>